In [1]:
import collections
import json
import keras
import numpy as np
import os
import sys
sys.path.append("../../../ecg")
import scipy.stats as sst

import util
import load

Using TensorFlow backend.


In [2]:
model_path = "/home/ubuntu/ecg/saved/cinc17/1572762595-293/0.419-0.853-013-0.360-0.871.hdf5"
data_path = "../dev.json"

data = load.load_dataset(data_path)
preproc = util.load(os.path.dirname(model_path))
model = keras.models.load_model(model_path)

100%|██████████| 852/852 [00:00<00:00, 3985.52it/s]



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [4]:
data_path = "../train.json"
with open("../train.json", 'r') as fid:
    train_labels = [json.loads(l)['labels'] for l in fid]
counts = collections.Counter(preproc.class_to_int[l[0]] for l in train_labels)
counts = sorted(counts.most_common(), key=lambda x: x[0])
counts = list(zip(*counts))[1]
smooth = 500
counts = np.array(counts)[None, None, :]
total = np.sum(counts) + counts.shape[1]
prior = (counts + smooth) / float(total)

In [5]:
probs = []
labels = []
for x, y  in zip(*data):
    x, y = preproc.process([x], [y])
    probs.append(model.predict(x))
    labels.append(y)

In [6]:
preds = []
ground_truth = []
for p, g in zip(probs, labels):
    preds.append(sst.mode(np.argmax(p / prior, axis=2).squeeze())[0][0])
    ground_truth.append(sst.mode(np.argmax(g, axis=2).squeeze())[0][0])

In [8]:
import sklearn.metrics as skm
report = skm.classification_report(
            ground_truth, preds,
            target_names=preproc.classes,
            digits=3)
scores = skm.precision_recall_fscore_support(
                    ground_truth,
                    preds,
                    average=None)
print(report)
print("CINC Average {:3f}".format(np.mean(scores[2][:3])))

              precision    recall  f1-score   support

           A      0.824     0.847     0.836        72
           N      0.915     0.885     0.900       486
           O      0.773     0.830     0.801       259
           ~      0.667     0.571     0.615        35

    accuracy                          0.852       852
   macro avg      0.795     0.783     0.788       852
weighted avg      0.854     0.852     0.852       852

CINC Average 0.845314
